# Markov Chain Monte Carlo for Spanning Trees 

<b> Click on the title and rename the file  with your name! </b>

In this exercise you have to implement the Markov-chain we defined on  the set of spanning trees of a graph G. Then you have to use this Markov-chain to estimate the probability that a given edge is in a random spanning tree, and finally you have to use it to estimate the number of spanning trees of the graph G.

Note that there is a fast direct way to both computing the probability an edge and count the number of spanning trees. Nevertheless, this homework is useful because we can see what can we expect from such an estimator and what cannot be expected. 
The framework we use is rather general, so we one can use it even when there is no clear alternative for exact computation.

# Function definitions

In [1]:
import numpy as np
import random

In [2]:
def SpanningTreeStep(G, A):
    """Given a graph G and a spanning tree A, it randomly selects one of its edges, delete it and randomly selects an
    edge from the resulting cut (maybe the original edge)."""
    # delete a random edge of spanning tree, result is a cut
    A.delete_edge(A.random_edge())
    # E(G) \ E(A - random edge):
    edges = list(set(G.edges())-set(A.edges()))
    # add an edge that connects the two components of the cut
    while edges:
        e = edges.pop()
        A.add_edges([e])
        if A.is_tree():
            return A
        else:
            A.delete_edge(e)

Next we use the random walk to estimate the probability of an edge.

In [3]:
def ProbabilityEstimator(G, e, iterator=5000):
    """Given a graph G and en edge e of it, we run the above SpanningTreeStep iterator many times to see how many 
    of the obtained spanning trees contain the edge e. The initial spanning tree can be chosen arbitrarily."""
    results = []
    for i in range(iterator):
        #arbitrary spanning tree of G
        A = G.random_spanning_tree(True)
        results.append(list(SpanningTreeStep(G=G, A=A).edges()).count(e))
    return float(sum(results) / iterator)

Next we use the above function to estimate the number of spanning trees. Let $\tau(G)$ be the number of spanning trees. Let $T$ be an arbitrary spanning tree of $G$, and let $E(G)\setminus E(T)=\{e_1,e_2,\dots ,e_{m-n+1}\}$. Then
$$\frac{1}{\tau(G)}=\prod_{i=1}^{m-n+1}\frac{\tau(G-e_1,\dots ,e_k)}{\tau(G-e_1,\dots ,e_{k-1})}.$$
Hence if $\textbf{T}$ denotes a random spanning tree, then
$$\tau(G)=\left(\prod_{i=1}^{m-n+1}\mathbb{P}_{G-e_1,\dots ,e_{i-1}}(e_i\notin \textbf{T})\right)^{-1}.$$
This can be estimated by repeated use of the previous function. (Please, make sure that you understand this expression!)

In [4]:
def SpanningTreeNumberEstimator(G, iterator=5000):
    """Using the above formula we can estimate the number of spanning trees of a graph G. 
    Since we can compute the number of spanning trees directly, we compare the two values. 
    The iterator number is the same iterator number that appears in the edge probability estimator. 
    So the algorithm will iterate (m-n+1)*iterator times, where m is the number of edges, n is the number of vertices."""
    T = G.random_spanning_tree(True)
    # E(G) \ E(T)
    edge_set = set(G.edges()) - set(T.edges())
    p = []
    B = G.copy()
    for e in edge_set:
        p.append(ProbabilityEstimator(G=B, e=e, iterator=iterator))
        B.delete_edge(e)
    p = np.array(p)
    estimator = int(round(np.prod(1/(1-p))))
    true_value = G.spanning_trees_count()   #it computes a determinant so it is very fast.
    ratio = N(estimator / true_value)
    return (estimator, true_value, ratio)

# Measurements

In [5]:
g=graphs.RandomRegular(3,20) #random 3-regular graph on 20 vertices
SpanningTreeNumberEstimator(g,1000)

(3809145, 3876852, 0.982535572675975)

In [6]:
g=graphs.RandomRegular(3,20) 
SpanningTreeNumberEstimator(g,5000)

(2696167, 4389753, 0.614195605083020)

In [7]:
g=graphs.RandomRegular(3,20) 
SpanningTreeNumberEstimator(g,20000)

(4920474, 3188864, 1.54301782703809)

In [8]:
g=graphs.RandomRegular(3,20) 
SpanningTreeNumberEstimator(g,50000)
#it will take a while, especially if your computer is weaker than mine.

(4492716, 3436635, 1.30730089171530)

In [9]:
g=graphs.RandomRegular(3,20) 
 
#run SpanningTreeNumberEstimator(g,10000)  10 times for the same graph and compare just the estimated 
#values to see whether you can expect a good estimate at all or not. 

#I strongly encourage you to make more experiments, especially if you have a strong enough computer.

n = 10
result = np.array([SpanningTreeNumberEstimator(g,10000) for _ in range(n)])
print(result)
print(result.min(axis=0))
print(result[:,-1])

[[8.67020400e+06 3.97239000e+06 2.18261651e+00]
 [3.74701300e+06 3.97239000e+06 9.43264131e-01]
 [4.57268900e+06 3.97239000e+06 1.15111784e+00]
 [2.17525000e+06 3.97239000e+06 5.47592256e-01]
 [3.06520000e+06 3.97239000e+06 7.71626149e-01]
 [7.41905500e+06 3.97239000e+06 1.86765524e+00]
 [1.24817180e+07 3.97239000e+06 3.14211797e+00]
 [3.12888500e+06 3.97239000e+06 7.87658060e-01]
 [3.30737300e+06 3.97239000e+06 8.32590204e-01]
 [7.69987400e+06 3.97239000e+06 1.93834795e+00]]
[2.17525000e+06 3.97239000e+06 5.47592256e-01]
[2.18261651 0.94326413 1.15111784 0.54759226 0.77162615 1.86765524
 3.14211797 0.78765806 0.8325902  1.93834795]
